In [4]:
import pandas as pd
pd.options.display.max_columns=200
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning, module='sklearn')

In [5]:
data = pd.read_csv('data/btc_data.csv', index_col=0)

In [6]:
data_crop = data.dropna()

In [7]:
# Sélectionner les caractéristiques et exclure la dernière ligne
features = data_crop.drop(columns=['progression tomorrow', 'target', 'close', 'high', 'low', 'volumefrom']).iloc[:-1, :]
target = data['target'].iloc[:-1]

In [8]:
len(data_crop)

3825

In [9]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler
import sys
import os

class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

In [10]:
"""# 1. Préparation des données

X = features.values
y = target.values

window_size = 1000 # Vous pouvez ajuster cette valeur selon vos besoins

# 2. Construction du modèle

model = keras.Sequential([
    layers.InputLayer(input_shape=(X.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 3. Entraînement du modèle avec validation walk-forward

for i in range(window_size, len(X) - 1):
    # Séparation des données en ensembles d'entraînement et de test
    X_train = X[i-window_size:i]
    y_train = y[i-window_size:i]
    X_test = X[i:i+1]
    y_test = y[i:i+1]

    # Normalisation des caractéristiques
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Entraînement du modèle
    model.fit(X_train, y_train, epochs=10, verbose=0)  # Vous pouvez ajuster le nombre d'epochs selon vos besoins

    # Évaluation du modèle (optionnel)
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"Step {i}, Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")
"""

'# 1. Préparation des données\n\nX = features.values\ny = target.values\n\nwindow_size = 1000 # Vous pouvez ajuster cette valeur selon vos besoins\n\n# 2. Construction du modèle\n\nmodel = keras.Sequential([\n    layers.InputLayer(input_shape=(X.shape[1],)),\n    layers.Dense(64, activation=\'relu\'),\n    layers.Dense(32, activation=\'relu\'),\n    layers.Dense(1, activation=\'sigmoid\')\n])\n\nmodel.compile(optimizer=\'adam\', loss=\'binary_crossentropy\', metrics=[\'accuracy\'])\n\n# 3. Entraînement du modèle avec validation walk-forward\n\nfor i in range(window_size, len(X) - 1):\n    # Séparation des données en ensembles d\'entraînement et de test\n    X_train = X[i-window_size:i]\n    y_train = y[i-window_size:i]\n    X_test = X[i:i+1]\n    y_test = y[i:i+1]\n\n    # Normalisation des caractéristiques\n    scaler = StandardScaler()\n    X_train = scaler.fit_transform(X_train)\n    X_test = scaler.transform(X_test)\n\n    # Entraînement du modèle\n    model.fit(X_train, y_train, e

In [11]:
# 1. Définir la métrique personnalisée

def earn_metric(predicted_probs, progressions, n_days):
    base = c = 1
    for j in range(n_days):
        index = len(predicted_probs) - n_days + j
        c *= predicted_probs[index] * progressions[index] + (1 - predicted_probs[index])
        base *= progressions[index]
    return c / base

In [12]:
"""
# Préparation des données

X = features.values
y = target.values

window_size = 2000  
n_days = 30 

# Construction du modèle

model = keras.Sequential([
    layers.InputLayer(input_shape=(X.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy')

# Entraînement du modèle avec validation walk-forward

predicted_probs = []
progressions = []
metrics = []

for i in range(window_size, len(X) - 1 - n_days):
    # Séparation des données
    X_train = X[i-window_size:i]
    y_train = y[i-window_size:i]
    X_test = X[i:i+n_days]
    y_test = y[i:i+n_days]

    # Normalisation
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    with HiddenPrints():
        # Entraînement
        model.fit(X_train, y_train, epochs=20, verbose=0)

        # Prédiction
        predicted_prob = model.predict(X_test).flatten()
        predicted_probs.extend(predicted_prob)

    # Récupérer la progression réelle
    progression = data_crop.iloc[i:i+n_days]['progression tomorrow'].values + 1
    progressions.extend(progression)

    # Calcul de la métrique personnalisée
    if len(predicted_probs) >= n_days:
        metric_value = earn_metric(predicted_probs, progressions, n_days)
        metrics.append(metric_value)
        #print(f"Step {i}, Earn Metric: {metric_value:.4f}")

print(f"Average Earn Metric: {np.mean(metrics):.4f}")"""

'\n# Préparation des données\n\nX = features.values\ny = target.values\n\nwindow_size = 2000  \nn_days = 30 \n\n# Construction du modèle\n\nmodel = keras.Sequential([\n    layers.InputLayer(input_shape=(X.shape[1],)),\n    layers.Dense(64, activation=\'relu\'),\n    layers.Dense(32, activation=\'relu\'),\n    layers.Dense(1, activation=\'sigmoid\')\n])\n\nmodel.compile(optimizer=\'adam\', loss=\'binary_crossentropy\')\n\n# Entraînement du modèle avec validation walk-forward\n\npredicted_probs = []\nprogressions = []\nmetrics = []\n\nfor i in range(window_size, len(X) - 1 - n_days):\n    # Séparation des données\n    X_train = X[i-window_size:i]\n    y_train = y[i-window_size:i]\n    X_test = X[i:i+n_days]\n    y_test = y[i:i+n_days]\n\n    # Normalisation\n    scaler = StandardScaler()\n    X_train = scaler.fit_transform(X_train)\n    X_test = scaler.transform(X_test)\n\n    with HiddenPrints():\n        # Entraînement\n        model.fit(X_train, y_train, epochs=20, verbose=0)\n\n     

In [13]:
#Average Earn Metric: 0.9799

In [14]:
"""import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler

# Préparation des données

X = features.values
y = target.values

window_size = 3000  
n_days = 30 

# Construction du modèle LSTM

model = keras.Sequential([
    layers.InputLayer(input_shape=(X.shape[1], 1)),  # Les LSTM attendent une entrée tridimensionnelle (batch_size, timesteps, features)
    layers.LSTM(64, return_sequences=True),  # Retourne une séquence à la couche suivante
    layers.LSTM(32),  # Retourne la dernière sortie de la séquence
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy')

# Entraînement du modèle avec validation walk-forward

predicted_probs = []
progressions = []
metrics = []

for i in range(window_size, len(X) - 1 - n_days):
    # Séparation des données
    X_train = X[i-window_size:i]
    y_train = y[i-window_size:i]
    X_test = X[i:i+n_days]
    y_test = y[i:i+n_days]

    # Normalisation
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Les LSTM attendent une entrée tridimensionnelle, donc nous devons remodeler les données
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

    with HiddenPrints():
        # Entraînement
        model.fit(X_train, y_train, epochs=10, verbose=0)

        # Prédiction
        predicted_prob = model.predict(X_test).flatten()
        predicted_probs.extend(predicted_prob)

    # Récupérer la progression réelle
    progression = data_crop.iloc[i:i+n_days]['progression tomorrow'].values + 1
    progressions.extend(progression)

    # Calcul de la métrique personnalisée
    if len(predicted_probs) >= n_days:
        metric_value = earn_metric(predicted_probs, progressions, n_days)
        metrics.append(metric_value)

print(f"Average Earn Metric: {np.mean(metrics):.4f}")"""

#2h15 pour un average metric à 1.0055... (epoch 10)

'import numpy as np\nfrom tensorflow import keras\nfrom tensorflow.keras import layers\nfrom sklearn.preprocessing import StandardScaler\n\n# Préparation des données\n\nX = features.values\ny = target.values\n\nwindow_size = 3000  \nn_days = 30 \n\n# Construction du modèle LSTM\n\nmodel = keras.Sequential([\n    layers.InputLayer(input_shape=(X.shape[1], 1)),  # Les LSTM attendent une entrée tridimensionnelle (batch_size, timesteps, features)\n    layers.LSTM(64, return_sequences=True),  # Retourne une séquence à la couche suivante\n    layers.LSTM(32),  # Retourne la dernière sortie de la séquence\n    layers.Dense(1, activation=\'sigmoid\')\n])\n\nmodel.compile(optimizer=\'adam\', loss=\'binary_crossentropy\')\n\n# Entraînement du modèle avec validation walk-forward\n\npredicted_probs = []\nprogressions = []\nmetrics = []\n\nfor i in range(window_size, len(X) - 1 - n_days):\n    # Séparation des données\n    X_train = X[i-window_size:i]\n    y_train = y[i-window_size:i]\n    X_test =

In [15]:
"""import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler

# Préparation des données
X = features.values
y = target.values

window_size = 3000  
n_days = 30 

best_metric = 0
best_params = None

# Espace de recherche
learning_rates = [0.001, 0.01, 0.1]
lstm_units = [32, 64, 128]

for lr in learning_rates:
    for units in lstm_units:
        # Construction du modèle
        model = keras.Sequential([
            layers.InputLayer(input_shape=(X.shape[1], 1)),
            layers.LSTM(units, return_sequences=True),
            layers.LSTM(units),
            layers.Dense(1, activation='sigmoid')
        ])
        
        optimizer = Adam(learning_rate=lr)
        model.compile(optimizer=optimizer, loss='binary_crossentropy')
        
        # Entraînement du modèle avec validation walk-forward
        predicted_probs = []
        progressions = []
        metrics = []

        for i in range(window_size, len(X) - 1 - n_days):
            # Séparation des données
            X_train = X[i-window_size:i]
            y_train = y[i-window_size:i]
            X_test = X[i:i+n_days]
            y_test = y[i:i+n_days]

            # Normalisation
            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)

            # Les LSTM attendent une entrée tridimensionnelle
            X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
            X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)


            with HiddenPrints():
                # Entraînement
                model.fit(X_train, y_train, epochs=10, verbose=0)

                # Prédiction
                predicted_prob = model.predict(X_test).flatten()
                predicted_probs.extend(predicted_prob)

            # Récupérer la progression réelle
            progression = data_crop.iloc[i:i+n_days]['progression tomorrow'].values + 1
            progressions.extend(progression)

            # Calcul de la métrique personnalisée
            if len(predicted_probs) >= n_days:
                metric_value = earn_metric(predicted_probs, progressions, n_days)
                metrics.append(metric_value)

        avg_metric = np.mean(metrics)
        
        if avg_metric > best_metric:
            best_metric = avg_metric
            best_params = {'learning_rate': lr, 'lstm_units': units}

print(f"Best metric: {best_metric:.4f}")
print(f"Best hyperparameters: {best_params}")"""


"""1000 minutes d'exec
Best metric: 1.0393
Best hyperparameters: {'learning_rate': 0.001, 'lstm_units': 32}"""

"1000 minutes d'exec\nBest metric: 1.0393\nBest hyperparameters: {'learning_rate': 0.001, 'lstm_units': 32}"

In [18]:
"""import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras.callbacks import EarlyStopping

# Préparation des données

X = features.values
y = target.values

window_size = 3000  
n_days = 30 

# Construction du modèle LSTM avec les hyperparamètres sélectionnés

model = keras.Sequential([
    layers.InputLayer(input_shape=(X.shape[1], 1)),  # Les LSTM attendent une entrée tridimensionnelle
    layers.LSTM(32, return_sequences=True),  # Utilisation de 32 unités comme déterminé par la recherche d'hyperparamètres
    layers.LSTM(32),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Définir l'early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Entraînement du modèle avec validation walk-forward et early stopping
predicted_probs = []
progressions = []
predicted_labels = []
true_labels = []


for i in range(window_size, len(X) - 1 - n_days):
    # Séparation des données
    X_train = X[i-window_size:i]
    y_train = y[i-window_size:i]
    X_test = X[i:i+n_days]
    y_test = y[i:i+n_days]

    # Normalisation
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Les LSTM attendent une entrée tridimensionnelle, donc nous devons remodeler les données
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

    with HiddenPrints():
        # Entraînement
        model.fit(X_train, y_train, epochs=30, verbose=0, validation_data=(X_test, y_test), callbacks=[early_stopping])

        # Prédiction
        predicted_prob = model.predict(X_test).flatten()
        predicted_label = [1 if prob > 0.5 else 0 for prob in predicted_prob]
        predicted_labels.extend(predicted_label)
        true_labels.extend(y_test)

# Calcul de l'accuracy
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Accuracy: {accuracy:.4f}")"""

"Accuracy 0.5369"

Accuracy: 0.5369
